In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

# importing files
- model file
- gender file
- geography file
- data scaling file

In [10]:
# load model file
model = load_model('model.h5')

# load gender file
with open('label_encoder_gender.pkl', 'rb') as file:
    gender_scaler = pickle.load(file)

# load geography file
with open('onehot_encoder.pkl', 'rb') as file:
    geography_encodeing = pickle.load(file)

# load scaler file
with open('scaler.pkl', 'rb') as file:
    data_scaler = pickle.load(file)

In [22]:
print(geography_encodeing)
print(gender_scaler)
print(data_scaler)

OneHotEncoder(sparse_output=False)
LabelEncoder()
StandardScaler()


In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
# converting input data into dataframe
data_frame = pd.DataFrame([input_data])
data_frame


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
# updating gender into 0,1
data_frame['Gender'] = gender_scaler.fit_transform(data_frame['Gender'])
data_frame

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


In [6]:
# updating Geography into one hot encodeing
droped = data_frame.drop('Geography', axis = 1)
droped


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,0,40,3,60000,2,1,1,50000


In [7]:
# converting Geography scamples into columns
geo_catagories = geography_scaler.get_feature_names_out(['Geography']) # it has 3 samples that are converted into features
geo_catagories


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [33]:
geo_encoded = geography_encodeing.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns= geo_catagories)
print(geo_encoded)
geo_encoded_df

[[1. 0. 0.]]


c:\Users\hp\Documents\Python\NLP_GenAi\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [8]:
data_frame

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


In [34]:
data_frame = pd.concat([droped, geo_encoded_df], axis=1)
data_frame


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
# scaling the input data
input_scaled = data_scaler.transform(data_frame)
input_scaled

array([[-0.47154541, -1.09997489,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [38]:
# model prediction
prediction_prob = model.predict(input_scaled)
prediction_prob


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step


array([[0.05841096]], dtype=float32)

In [44]:
print(prediction_prob[0][0])

0.058410965


In [39]:
if prediction_prob > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
